# Out-of-Distribution (OOD) Detection Tutorial


## _Problem Statement_

For most computer vision tasks like **image classification** and **object detection**, out-of-distribution (OOD) detection can provide insight into operational drift, or training problems. A way to identify these is through autoencoding reconstruction error.

To help with this, DataEval has an OOD detector that allows a user to identify these images.


### _When to use_

The `OOD_AE` class and similar should be used when you would like to find individual images in a dataset which are the most different from the others in the provided set.


### _What you will need_

1. A training image dataset with the approximate percentage of known OOD images.
2. A test image dataset to evaluate for OOD images.
3. A python environment with the following packages installed:
   - `dataeval` or `dataeval[all]`


### _Setting up_

Let's import the required libraries needed to set up a minimal working example


In [ ]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval
except Exception:
    pass

import numpy as np
import torch

np.random.seed(0)
torch.manual_seed(0)

In [2]:
import numpy as np

from dataeval.detectors.ood import OOD_AE
from dataeval.utils.dataset.datasets import MNIST
from dataeval.utils.torch.models import AE

## Load the data

We will use the MNIST dataset for this tutorial


In [ ]:
# Load in the training MNIST dataset and use the first 2000
train_ds = MNIST(root="./data/", train=True, download=True, size=2000, unit_interval=True, channels="channels_first")

# Split out the images and labels
images, labels = train_ds.data, train_ds.targets
input_shape = images[0].shape

## Initialize the model

Now, lets look at how to use DataEval's OOD detection methods.
We will focus on a simple autoencoder network from our Alibi Detect provider


In [4]:
OOD_detectors = [
    OOD_AE(AE(input_shape=input_shape)),
]

## Train the model

Next we will train a model on the dataset.
For better results, the epochs can be increased.
We set the threshold to detect the most extreme 1% of training data as out-of-distribution.


In [ ]:
for detector in OOD_detectors:
    print(f"Training {detector.__class__.__name__}...")
    detector.fit(images, threshold_perc=99, epochs=23)

## Test for OOD

We have trained our detector on a dataset of digits.
What happens when we give it corrupted images of digits, which we expect to be OOD?


In [ ]:
corruption = MNIST(
    root="./data",
    train=True,
    download=False,
    size=2000,
    unit_interval=True,
    channels="channels_first",
    corruption="translate",
)
corrupted_images = corruption.data

Now we evaluate the two datasets using the trained model.


In [ ]:
results = {type(detector).__name__: np.mean(detector.predict(images).is_ood) for detector in OOD_detectors}
print(results)

In [ ]:
corrupted = {type(detector).__name__: np.mean(detector.predict(corrupted_images).is_ood) for detector in OOD_detectors}
print(corrupted)

In [9]:
### TEST ASSERTION CELL ###
assert results["OOD_AE"] < 0.05
# Inconsistent result - uncomment when OOD_VAE implementation is complete
# assert corrupted["OOD_AE"] > 0.65

### Results

We can see that the Autoencoder based OOD detector was able to identify most of the translated images as outliers.

Depending on your needs, certain outlier detectors will work better than others under specific conditions.
